# Introduction to the Keras Tuner

## Overview

The Keras Tuner is a library that helps you pick the optimal set of hyperparameters for your TensorFlow program. The process of selecting the right set of hyperparameters for your machine learning (ML) application is called *hyperparameter tuning* or *hypertuning*.

Hyperparameters are the variables that govern the training process and the topology of an ML model. These variables remain constant over the training process and directly impact the performance of your ML program. Hyperparameters are of two types:
1. **Model hyperparameters** which influence model selection such as the number and width of hidden layers
2. **Algorithm hyperparameters** which influence the speed and quality of the learning algorithm such as the learning rate for Stochastic Gradient Descent (SGD) and the number of nearest neighbors for a k Nearest Neighbors (KNN) classifier

In this tutorial, you will use the Keras Tuner to perform hypertuning for an image classification application.

## Setup

In [1]:
import tensorflow as tf
from tensorflow import keras

Install and import the Keras Tuner.

In [2]:
!pip install -q -U keras-tuner

In [3]:
import keras_tuner as kt

## Download and prepare the dataset

In this tutorial, you will use the Keras Tuner to find the best hyperparameters for a machine learning model that classifies images of clothing from the [Fashion MNIST dataset](https://github.com/zalandoresearch/fashion-mnist).

Load the data.

In [4]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [5]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

## Define the model

When you build a model for hypertuning, you also define the hyperparameter search space in addition to the model architecture. The model you set up for hypertuning is called a *hypermodel*.

You can define a hypermodel through two approaches:

* By using a model builder function
* By subclassing the `HyperModel` class of the Keras Tuner API

You can also use two pre-defined `HyperModel` classes - [HyperXception](https://keras-team.github.io/keras-tuner/documentation/hypermodels/#hyperxception-class) and [HyperResNet](https://keras-team.github.io/keras-tuner/documentation/hypermodels/#hyperresnet-class) for computer vision applications.

In this tutorial, you use a model builder function to define the image classification model. The model builder function returns a compiled model and uses hyperparameters you define inline to hypertune the model.

In [11]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=28, max_value=40, step=2)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), ##Whether y_pred is expected to be a logits tensor. By default, we assume that y_pred encodes a probability distribution.
                metrics=['accuracy'])

  return model

## Instantiate the tuner and perform hypertuning

Instantiate the tuner to perform the hypertuning. The Keras Tuner has four tuners available - `RandomSearch`, `Hyperband`, `BayesianOptimization`, and `Sklearn`. In this tutorial, you use the [Hyperband](https://arxiv.org/pdf/1603.06560.pdf) tuner.

To instantiate the Hyperband tuner, you must specify the hypermodel, the `objective` to optimize and the maximum number of epochs to train (`max_epochs`).

In [20]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=2,
                     directory='my_dir',
                     project_name='intro_to_kt',
                     overwrite=True) ## so the logs get overwritten for everY manual tuning

The my_dir/intro_to_kt directory contains detailed logs and checkpoints for every trial (model configuration) run during the hyperparameter search. If you re-run the hyperparameter search, the Keras Tuner uses the existing state from these logs to resume the search. To disable this behavior, pass an additional overwrite=True argument while instantiating the tuner.

The Hyperband tuning algorithm uses adaptive resource allocation and early-stopping to quickly converge on a high-performing model. This is done using a sports championship style bracket. The algorithm trains a large number of models for a few epochs and carries forward only the top-performing half of models to the next round. Hyperband determines the number of models to train in a bracket by computing 1 + log<sub>`factor`</sub>(`max_epochs`) and rounding it up to the nearest integer.

Stopping Criteria:
Here are some of the possible ways to terminate the algorithm:

Terminate the algorithm once a specified number of iterations is satisfied.

Terminate the algorithm once a specified MSE is satisfied.

Terminate the algorithm if the MSE/Val_acc/loss does not improve in the next iteration. You can specify a precision such as 0.001, if the difference between two successive MSEs/Val_accs/losses is less than this precision, then stop the algorithm.

Create a callback to stop training early after reaching a certain value for the validation loss.

In [13]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
## monitor - the quantity to be monitored
## patience - Number of epochs with no improvement after which training will be stopped.
## so here, if in 5 epochs no change in value - stop

Run the hyperparameter search. The arguments for the search method are the same as those used for `tf.keras.model.fit` in addition to the callback above.

In [14]:
tuner.search(img_train, label_train, epochs=2, validation_split=0.2, callbacks=[stop_early])

Trial 3 Complete [00h 00m 46s]
val_accuracy: 0.8606666922569275

Best val_accuracy So Far: 0.8896666765213013
Total elapsed time: 00h 02m 22s
INFO:tensorflow:Oracle triggered exit


## Train the model

Find the optimal number of epochs to train the model with the hyperparameters obtained from the search.

In [15]:
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")


The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 384 and the optimal learning rate for the optimizer
is 0.001.



In [16]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=5, validation_split=0.2)
## have to do with large number of epoch to select best epoch ideally give 50

Epoch 1/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.4939 - accuracy: 0.8248 - val_loss: 0.3976 - val_accuracy: 0.8575
Epoch 2/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3695 - accuracy: 0.8657 - val_loss: 0.3505 - val_accuracy: 0.8758
Epoch 3/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3276 - accuracy: 0.8809 - val_loss: 0.3643 - val_accuracy: 0.8690
Epoch 4/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3050 - accuracy: 0.8874 - val_loss: 0.3319 - val_accuracy: 0.8777
Epoch 5/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2849 - accuracy: 0.8945 - val_loss: 0.3185 - val_accuracy: 0.8862


In [17]:
val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Best epoch: 5


Re-instantiate the hypermodel and train it with the optimal number of epochs from above.

In [18]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.4933 - accuracy: 0.8243 - val_loss: 0.4270 - val_accuracy: 0.8457
Epoch 2/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3701 - accuracy: 0.8654 - val_loss: 0.3746 - val_accuracy: 0.8619
Epoch 3/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3300 - accuracy: 0.8795 - val_loss: 0.3407 - val_accuracy: 0.8738
Epoch 4/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3063 - accuracy: 0.8861 - val_loss: 0.3492 - val_accuracy: 0.8752
Epoch 5/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2857 - accuracy: 0.8942 - val_loss: 0.3347 - val_accuracy: 0.8794


To finish this tutorial, evaluate the hypermodel on the test data.

In [19]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

313/313 [==============================] - 1s 3ms/step - loss: 0.3636 - accuracy: 0.8679
[test loss, test accuracy]: [0.3636450171470642, 0.867900013923645]


## Summary

In this tutorial, you learned how to use the Keras Tuner to tune hyperparameters for a model. To learn more about the Keras Tuner, check out these additional resources:

* [Keras Tuner on the TensorFlow blog](https://blog.tensorflow.org/2020/01/hyperparameter-tuning-with-keras-tuner.html)
* [Keras Tuner website](https://keras-team.github.io/keras-tuner/)

Also check out the [HParams Dashboard](https://www.tensorflow.org/tensorboard/hyperparameter_tuning_with_hparams) in TensorBoard to interactively tune your model hyperparameters.